local_filename = preprocess.preprocess_region_t(t, sensor="VIIRS", region=region)## One Month Fire Forward

Given that you already have all the preprocessed_region_t files on s3, now read them in and run `Fire_Forward`.

In [ ]:
# If you haven't installed the fireatlas code yet, uncomment the following line and run this cell.

# !pip install -e .. -q

# After this runs, restart the notebook kernel.

In [1]:
import s3fs
from fireatlas import postprocess, FireMain, FireIO

tst = [2023, 1, 1, 'AM']
ted = [2023, 2, 1, 'AM']
region = ('WesternUS',[-125.698046875,31.676476158707615,
                       -101.00078125,49.51429477264348])

2024-04-03 09:57:21,460 - fireatlas.FireLog - INFO - logger initialized!


This is how you run `Fire_Forward` for many timesteps. 

Beware: even a month for a large area like this will take a long time (up to a few hours)

In [2]:
%%time
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, region=region, restart=False, read_location="s3", read_saved_location="local")

2024-04-03 08:00:19,103 - fireatlas.FireLog - INFO - func:read_preprocessed took: 1.26 sec
2024-04-03 08:00:19,157 - fireatlas.FireLog - INFO - func:read_preprocessed took: 53.19 ms
2024-04-03 08:00:19,227 - fireatlas.FireLog - INFO - func:read_preprocessed took: 69.16 ms
2024-04-03 08:00:19,276 - fireatlas.FireLog - INFO - func:read_preprocessed took: 48.71 ms
2024-04-03 08:00:19,319 - fireatlas.FireLog - INFO - func:read_preprocessed took: 41.93 ms
2024-04-03 08:00:19,362 - fireatlas.FireLog - INFO - func:read_preprocessed took: 43.05 ms
2024-04-03 08:00:19,411 - fireatlas.FireLog - INFO - func:read_preprocessed took: 48.25 ms
2024-04-03 08:00:19,449 - fireatlas.FireLog - INFO - func:read_preprocessed took: 37.41 ms
2024-04-03 08:00:19,498 - fireatlas.FireLog - INFO - func:read_preprocessed took: 48.42 ms
2024-04-03 08:00:19,544 - fireatlas.FireLog - INFO - func:read_preprocessed took: 44.88 ms
2024-04-03 08:00:19,598 - fireatlas.FireLog - INFO - func:read_preprocessed took: 53.87 ms

KeyboardInterrupt: 

Now we don't necessarily need to store all the allpixels and allfires dataframes in the s3 bucket. It's enough to put files for the last timestep.

In [2]:
last_saved_t = postprocess.get_t_of_last_allfires_run(tst, ted, region, location="local")

allpixels_filepath = postprocess.allpixels_filepath(tst, last_saved_t, region, location="local")
allfires_filepath = postprocess.allfires_filepath(tst, last_saved_t, region, location="local")

In [3]:
FireIO.copy_from_local_to_s3(allpixels_filepath)
FireIO.copy_from_local_to_s3(allfires_filepath)

2024-04-03 09:57:23,054 - fireatlas.FireLog - INFO - uploading file /projects/fireatlas_nrt/data/FEDSoutput-v3/WesternUS/2023/allpixels_20230119_PM.csv to s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-v3/WesternUS/2023/allpixels_20230119_PM.csv
2024-04-03 09:57:24,399 - fireatlas.FireLog - INFO - uploading file /projects/fireatlas_nrt/data/FEDSoutput-v3/WesternUS/2023/allfires_20230119_PM.parq to s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-v3/WesternUS/2023/allfires_20230119_PM.parq


In [8]:
remote_filepath = postprocess.allpixels_filepath(tst, last_saved_t, region, location="s3")

In [9]:
s3 = s3fs.S3FileSystem()
s3.get_tags(remote_filepath)

{'epsg': '9311',
 'processedBy': 'jsignell',
 'remove_static_sources_bool': 'True'}